In [369]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv'
my_submission_path = 'submission.csv'

puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)

In [370]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_1/8']
# Two types: permutation & colors

,id,puzzle_type,solution_state,initial_state,num_wildcards
388,388,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,G;Q;G;D;E;T;E;N;U;H;H;U;O;F;T;B;S;M;A;N;D;V;V;...,6
389,389,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,H;A;P;K;G;N;U;G;D;V;E;L;D;O;E;Q;D;A;A;H;L;J;A;...,0
390,390,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,F;O;N;B;B;D;V;E;B;J;F;L;S;G;U;E;O;P;U;F;R;R;J;...,0
391,391,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N237;N257;N199;N215;N54;N261;N50;N6;N57;N32;N2...,0
392,392,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,L;A;O;U;T;Q;F;P;B;V;H;T;O;C;J;P;V;P;N;C;T;V;G;...,0
393,393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0


In [371]:
idx = 343

N = 1+1
M = int(2*8)
apply_move = {}

# Solving 2xM

In [81]:
def print_globe(b):
    print(np.asarray(b).reshape(2, M))

In [373]:
def make_moves():
    """
    Creates global variable apply_move
    """
    
    # Create perms
    r0_perm = np.array([i for i in range(2*M)])
    r1_perm = np.array([i for i in range(2*M)])
    f0_perm = np.array([i for i in range(2*M)])

    r0_perm[M-1] = 0
    for i in range(0, M-1):
        r0_perm[i] = i+1

    r1_perm[2 * M - 1] = M
    for i in range(M, 2*M-1):
        r1_perm[i] = i+1

    for i in range(M//2):
        f0_perm[i] = M + M//2 - 1 - i

    for i in range(M, M + M//2):
        f0_perm[i] = M//2-1 - (i-M)

    nr0_perm = np.array([i for i in range(2*M)])
    nr1_perm = np.array([i for i in range(2*M)])

    for i in range(M-1):
        nr0_perm = nr0_perm[r0_perm]

    for i in range(M-1):
        nr1_perm = nr1_perm[r1_perm]
        
    # Create r moves
    
    def r0(arr, moves):
        moves.append('r0')
        arr = arr[r0_perm]
        return arr

    def nr0(arr, moves):
        moves.append('-r0')
        arr = arr[nr0_perm]
        return arr

    def r1(arr, moves):
        moves.append('r1')
        arr = arr[r1_perm]
        return arr

    def nr1(arr, moves):
        moves.append('-r1')
        arr = arr[nr1_perm]
        return arr

    def f0(arr, moves):
        moves.append('f0')
        arr = arr[f0_perm]
        return arr

    apply_move['r0'] = r0
    apply_move['-r0'] = nr0
    apply_move['r1'] = r1
    apply_move['-r1'] = nr1
    apply_move['f0'] = f0
    
    # Create f moves
    
    def create_f_function(i, cur_perm):
        return lambda arr, moves: (moves.append('f' + str(i)), arr[cur_perm])[1]

    for i in range(1, M):
        cur_perm = np.array([i for i in range(2*M)])

        for j in range(i):
            cur_perm = cur_perm[r0_perm]
            cur_perm = cur_perm[r1_perm]

        cur_perm = cur_perm[f0_perm]

        for j in range(i):
            cur_perm = cur_perm[nr0_perm]
            cur_perm = cur_perm[nr1_perm]

        apply_move['f' + str(i)] = create_f_function(i, cur_perm)

r0
[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
   19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
   37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
   55  56  57  58  59  60  61  62  63  64  65   0]
 [ 66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
   84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
  102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
  120 121 122 123 124 125 126 127 128 129 130 131]]
nr0
[[ 65   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
   17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
   35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
   53  54  55  56  57  58  59  60  61  62  63  64]
 [ 66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
   84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
  102 103 104 105 106 107

In [374]:
make_moves(M)

208
823
[['A' 'A' 'A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'Q' 'Q'
  'Q' 'Q' 'Q' 'S' 'S' 'S' 'S' 'S' 'S' 'U' 'U' 'U' 'U' 'U' 'U' 'N' 'J' 'H'
  'K' 'I' 'O' 'N' 'K' 'O' 'H' 'K' 'P' 'K' 'F' 'O' 'H' 'E' 'J' 'P' 'H' 'H'
  'J' 'L' 'E' 'N' 'R' 'P' 'M' 'P' 'G' 'N' 'J' 'I']
 ['B' 'B' 'B' 'B' 'B' 'B' 'D' 'D' 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'R' 'R'
  'R' 'R' 'R' 'T' 'T' 'T' 'T' 'T' 'T' 'V' 'V' 'V' 'V' 'V' 'V' 'O' 'J' 'G'
  'F' 'I' 'I' 'M' 'G' 'I' 'N' 'L' 'L' 'N' 'K' 'P' 'O' 'I' 'G' 'J' 'P' 'O'
  'G' 'M' 'L' 'M' 'L' 'Q' 'G' 'M' 'M' 'L' 'H' 'K']]
[['A' 'A' 'A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'E' 'E'
  'G' 'G' 'G' 'G' 'G' 'G' 'I' 'I' 'I' 'I' 'I' 'I' 'K' 'K' 'K' 'K' 'K' 'K'
  'M' 'M' 'M' 'M' 'M' 'M' 'O' 'O' 'O' 'O' 'O' 'O' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q'
  'S' 'S' 'S' 'S' 'S' 'S' 'U' 'U' 'U' 'U' 'U' 'U']
 ['B' 'B' 'B' 'B' 'B' 'B' 'D' 'D' 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'F' 'F'
  'H' 'H' 'H' 'H' 'H' 'H' 'J' 'J' 'J' 'J' 'J' 'J' 'L' 'L' 'L' 'L' 'L' 'L'
  'N' 'N' 'N' 'N' 'N' 'N

In [375]:
def half_turn_ccw(arr, moves):
    for i in range(M//2):
        arr = apply_move['-r0'](arr, moves)
    
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    
    for i in range(M//2):
        arr = apply_move['r0'](arr, moves)
    return arr

print_globe(half_turn_ccw(test_globe, []))

def half_turn_cw(arr, moves):
    for i in range(M//2):
        arr = apply_move['-r0'](arr, moves)
    
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['-r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    
    for i in range(M//2):
        arr = apply_move['r0'](arr, moves)
    return arr

apply_move['half_turn'] = half_turn_ccw
apply_move['rev_half_turn'] = half_turn_cw

print_globe(half_turn_cw(test_globe, []))

In [376]:
test_globe = np.array([i for i in range(2*M)])

def top_right_cw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['-r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

def top_right_ccw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

def top_left_ccw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['-r1'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

def top_left_cw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['r1'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

apply_move['top_right_cw'] = top_right_cw
apply_move['top_right_ccw'] = top_right_ccw
apply_move['top_left_cw'] = top_left_cw
apply_move['top_left_ccw'] = top_left_ccw

print_globe(top_right_cw(test_globe, []))
print_globe(top_right_ccw(test_globe, []))
print_globe(top_left_cw(test_globe, []))
print_globe(top_left_ccw(test_globe, []))

## Phase 1: Sort Elements Into Top/Bottom Arrays

In [52]:
test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx]
test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
print_globe(test_globe)

In [ ]:
moves = []

In [90]:
def is_moved_top_bottom(arr):
    for i in range(M):
        if arr[i] >= M:
            return False
    for i in range(M, 2*M):
        if arr[i] < M:
            return False
    return True

# change this to allow all M cyclic rotates
def move_top_bottom_helper(arr, moves):
    while not is_moved_top_bottom(arr):
        
        rotate_locs = [
            (M//2, M + M // 2 - 1, 'top_right_ccw'),
            (M-1, M, 'top_right_cw'),
            (M//2-1, M + M//2, 'top_left_cw'),
            (0, 2*M-1, 'top_left_ccw')
        ]

        all_costs = []

        for a, b, op_name in rotate_locs:
            mn_a, mn_b = int(1e9), int(1e9)
            top_ops, bot_ops = [], []
            
            for i in range(M):
                if arr[i] >= M: # TODO: Try no abs. Can only r0
                    if abs(i - a) < mn_a:
                        mn_a = abs(i-a)

                        if i < a: # rotate right
                            top_ops = ['-r0'] * (a - i)
                        else:
                            top_ops = ['r0'] * (i - a)

            for i in range(M, 2*M):
                if arr[i] < M:
                    if abs(i - b) < mn_b:
                        mn_b = abs(i - b)

                        if i < b:
                            bot_ops = ['-r1'] * (b - i)
                        else:
                            bot_ops = ['r1'] * (i - b)

            all_costs.append((mn_a + mn_b, top_ops + bot_ops, op_name))

        all_costs = list(sorted(all_costs))
        for i in all_costs[0][1]:
            arr = apply_move[i](arr, moves)
        arr = apply_move[all_costs[0][2]](arr, moves)
    return arr

print_globe(move_top_bottom_helper(test_globe, []))

[[13  1  9  8 11  2  4  5  6  3  7 10 14  0 12 15]
 [28 21 31 18 19 29 22 24 27 23 26 16 25 17 30 20]]


In [91]:
# def sort_top_bottom(arr, moves):
#     a = []
#     a_arr = sort_top_bottom_helper(arr, a)

#     b = ['f0', 'f' + str(M//2)] # untested
#     arr = apply_move['f0'](arr, [])
    
#     for i in range(M//2):
#         arr = apply_move['r0'](arr, [])
#         arr = apply_move['r1'](arr, [])

#     arr = apply_move['f0'](arr, [])

#     for i in range(M//2):
#         arr = apply_move['-r0'](arr, [])
#         arr = apply_move['-r1'](arr, [])

#     b_arr = sort_top_bottom_helper(arr, b)

#     if len(b) < len(a):
#         for i in b:
#             moves.append(i)
#         return b_arr

#     for i in a:
#         moves.append(i)
#     return a_arr

# test_globe = sort_top_bottom(test_globe, moves)
# test_globe = sort_top_bottom_helper(test_globe, moves)
# print_globe(test_globe)

In [92]:
test_globe = move_top_bottom_helper(test_globe, moves)
top_bot_moves = moves
top_bot_state = test_globe
# try only applying ex-red here

# Phase 2: Remove Inversions

In [93]:
test_globe = top_bot_state
moves = top_bot_moves

print_globe(test_globe)

[[13  1  9  8 11  2  4  5  6  3  7 10 14  0 12 15]
 [28 21 31 18 19 29 22 24 27 23 26 16 25 17 30 20]]


In [94]:
def merge_and_count(arr, temp, left, mid, right):
    i, j, k = left, mid + 1, left
    inv_count = 0

    while i <= mid and j <= right:
        if arr[i] <= arr[j]:
            temp[k] = arr[i]
            i += 1
        else:
            temp[k] = arr[j]
            inv_count += (mid - i + 1)
            j += 1
        k += 1

    while i <= mid:
        temp[k] = arr[i]
        k += 1
        i += 1

    while j <= right:
        temp[k] = arr[j]
        k += 1
        j += 1

    for i in range(left, right + 1):
        arr[i] = temp[i]

    return inv_count

def merge_sort_and_count(arr, temp, left, right):
    inv_count = 0
    if left < right:
        mid = (left + right) // 2
        inv_count += merge_sort_and_count(arr, temp, left, mid)
        inv_count += merge_sort_and_count(arr, temp, mid + 1, right)
        inv_count += merge_and_count(arr, temp, left, mid, right)
    return inv_count

def count_inversions(arr):
    n = len(arr)
    temp_arr = arr.copy()  # Create a copy of the array
    temp = [0] * n
    return merge_sort_and_count(temp_arr, temp, 0, n - 1)

print(count_inversions(test_globe[0:M]))
print(count_inversions(test_globe[M:2*M]))

49
66


In [95]:
temp_test_globe = np.array([i for i in range(2*M)])

# swaps i, i+1 for both top and bottom
def swap_same_idx(i, arr, moves):
    loc = M // 2 - 1
    if i < loc:
        i += M

    delta = i - loc

    arr = apply_move['f' + str(delta)](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f' + str(delta)](arr, moves)

    arr = apply_move['r1'](arr, moves)
    arr = apply_move['r1'](arr, moves)

    plus_seven = (delta + 7) % M

    arr = apply_move['f' + str(plus_seven)](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f' + str(plus_seven)](arr, moves)

    arr = apply_move['-r0'](arr, moves) # cuold remove, but should be handled by ex-red
    arr = apply_move['-r1'](arr, moves) # cuold remove, but should be handled by ex-red

    return arr

print_globe(swap_same_idx(1, temp_test_globe, []))

[[ 0  2  1  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 18 17 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [96]:
def get_inversion_locs(arr):
    A, B = [], []
    for i in range(0, M-1):
        if (arr[i] > arr[i+1]):
            A.append(i)

    for i in range(M, 2*M-1):
        if (arr[i] > arr[i+1]):
            B.append(i - M)

    return (A, B)

# Could try: Non-circular min_abs_diff
def min_abs_diff(A, B):
    A = A + [i + M for i in A]
    B = B + [i + M for i in B]
    
    i, j = 0, 0
    min_diff = int(1e9)
    min_indices = (-1, -1)

    while i < len(A) and j < len(B):
        diff = abs(A[i] - B[j])

        if diff < min_diff:
            min_diff = diff
            min_indices = (A[i], B[j])

        if A[i] < B[j]:
            i += 1
        else:
            j += 1

    if min_indices[0] >= M:
        min_indices = (min_indices[0] - M, min_indices[1])

    if min_indices[1] >= M:
        min_indices = (min_indices[0], min_indices[1] - M)
        
    return min_indices

def is_array_zero_to_n(arr):
    n = len(arr) - 1
    return len(arr) == n + 1 and set(arr) == set(range(n + 1))

def check_swaps(arr, swaps):
    arr_copy = arr.copy()
    for swap in swaps:
        if 0 <= swap < len(arr_copy) - 1:
            arr_copy[swap], arr_copy[swap + 1] = arr_copy[swap + 1], arr_copy[swap]
            # print(arr_copy)
        else:
            return False
    return arr_copy == sorted(arr_copy)

# Phase 2, finding optimal order of swaps of one array to minimize the amount of rotations of other
# Minimize sum of absolute distances between swaps
def get_swaps_order(arr):
    swaps = []
    arr = [i % M for i in arr]

    mx = -1
    move_forward = []
    for i in range(len(arr)):
        if(arr[i] < mx):
             move_forward.append(i)
        mx = max(mx, arr[i])

    while move_forward:
        for i in range(len(move_forward)-1, -1, -1):
            if i != 0 and move_forward[i-1] == move_forward[i] - 1:
                continue

            swaps.append(move_forward[i] - 1)
            arr[move_forward[i]-1], arr[move_forward[i]] = arr[move_forward[i]], arr[move_forward[i]-1]
            move_forward[i] -= 1
            
            temp = -1
            for k in range(move_forward[i]):
                temp = max(temp, arr[k])

            
            if temp <= arr[move_forward[i]]:
                move_forward.pop(i)
                
            break

    if (not is_array_zero_to_n(arr)):
        print("get_swaps_order function is incorrect")
        # swaps = swaps + get_swaps_order(arr)
    
    return swaps

x = get_swaps_order([2,6,0,5,4,7,1,3])

In [97]:
def is_array_zero_to_n(arr):
    n = len(arr) - 1
    return len(arr) == n + 1 and set(arr) == set(range(n + 1))

def remove_inversions(arr, moves):
    # print("A")
    # print_globe(arr)

    # Phase 1: Both Have Inversions
    up_inversions, down_inversions = get_inversion_locs(arr)

    while up_inversions and down_inversions:
        up_i, down_i = min_abs_diff(up_inversions, down_inversions)

        if down_i < up_i: 
            diff = up_i - down_i # Could do: Try swapping K/2 left up, K/2 down right
            for k in range(diff):
                arr = apply_move['r0'](arr, moves)

            arr = swap_same_idx(down_i, arr, moves)

            for k in range(diff):
                arr = apply_move['-r0'](arr, moves)

        else:
            diff = down_i - up_i # Could do: Try swapping K/2 left up, K/2 down right
            for k in range(diff):
                arr = apply_move['r1'](arr, moves)

            arr = swap_same_idx(up_i, arr, moves)

            for k in range(diff):
                arr = apply_move['-r1'](arr, moves)

        up_inversions, down_inversions = get_inversion_locs(arr)
        

    # print("B")
    # print_globe(arr)
    
    # Phase 2: Only one has inversion (FIX)
    if (count_inversions(arr[M:2*M]) + count_inversions(arr[0:M])) % 2 != 0:
        print("Failure: Leftover Inversions should be even")
        return [-1]

    if up_inversions:
        up_swaps = get_swaps_order(arr[0:M])
        for k in range(0, len(up_swaps), 2):
            diff = (up_swaps[k] - up_swaps[k+1] + M) % M
            
            arr = swap_same_idx(up_swaps[k], arr, moves)
            
            for l in range(diff):
                arr = apply_move['r1'](arr, moves)
            arr = swap_same_idx(up_swaps[k+1], arr, moves)
            
        
    if down_inversions:
        down_swaps = get_swaps_order(arr[M:2*M])
        for k in range(0, len(down_swaps), 2):
            diff = (down_swaps[k] - down_swaps[k+1] + M) % M
            
            arr = swap_same_idx(down_swaps[k], arr, moves)
            
            for l in range(diff):
                arr = apply_move['r0'](arr, moves)

            arr = swap_same_idx(down_swaps[k+1], arr, moves)


    # Phase 3: Do some final rotations to get everything aligned right. Check correctness
    while arr[0] != 0:
        arr = apply_move['r0'](arr, moves)

    while arr[M] != M:
        arr = apply_move['r1'](arr, moves)
    
    # print("C")
    # print_globe(arr)

    return arr


# test_globe = apply_move['r0'](test_globe, [])

print_globe(test_globe)
remove_inversions(test_globe, [])

[[13  1  9  8 11  2  4  5  6  3  7 10 14  0 12 15]
 [28 21 31 18 19 29 22 24 27 23 26 16 25 17 30 20]]
Failure: Leftover Inversions should be even


[-1]

In [98]:
def sort_top_bot(arr):
    res = [-1]

    for i in range(M-1):
        for j in range(M-1):
            cur_moves = []
            cur_arr = arr
            
            for k in range(i):
                cur_arr = apply_move['r0'](cur_arr, cur_moves)

            for k in range(j):
                cur_arr = apply_move['r1'](cur_arr, cur_moves)
            
            if (count_inversions(cur_arr[0:M]) % 2 != count_inversions(cur_arr[M:2*M]) % 2):
                continue

            cur_arr = remove_inversions(cur_arr, cur_moves)

            # TODO: Apply Exp-Red

            if -1 in res or len(cur_moves) < len(res):
                res = cur_moves
    
    if -1 in res:
        print("Failure: No Rotations where top and bottom have same parity inversions")

    return res

res = sort_top_bot(test_globe)

In [99]:
test_globe_copy = test_globe
for i in res:
    test_globe_copy = apply_move[i](test_globe_copy, [])
print_globe(test_globe_copy)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [100]:
print(len(res))

623


In [101]:
moves = moves + res
print(len(moves))

667


In [102]:
test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx]
test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
print_globe(test_globe)

[[21  1  9 20 30  8 25 26 11  2 16 27  4 28  6 13]
 [17 31 12 18 19  0 29 10 22 14 24  7  3  5 15 23]]


In [103]:
for i in moves:
    test_globe = apply_move[i](test_globe, [])
print_globe(test_globe)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [104]:
def expand(path, n = 2, m = 16):    
    rows = [[] for _ in range(n)]
    
    def expand_f(x):
        for i in range(n):
            rows[i].append(x)
            rows[i].append('f0')
            rows[i].append(m - x)
    
    for i in path:
        if 'f' in i:
            if '-' in i:
                expand_f(int(i[2:]))
            else:
                expand_f(int(i[1:]))
        else:
            for j in range(n):
                temp = 'r' + str(j)

                if temp == i:
                    rows[j].append(1)
                    
                temp = '-r' + str(j)

                if temp == i:
                    rows[j].append(m-1)

    Rrows = [[] for _ in range(n)]

    for r in range(n):
        run = 0
        for i in rows[r]:
            if type(i) == type(0):
                run += i
            else:
                Rrows[r].append(run)
                Rrows[r].append('f0')
                run = 0
            
        Rrows[r].append(run)
    
    return Rrows

def shrink(rows, n = 2, m = 16):
    for i in range(len(rows[0])):
        mn = int(1e9)
        for j in range(n):
            if type(rows[j][i]) == type(1):
                rows[j][i] %= m
            else:
                mn = min(mn, rows[j][i-1])
        
        for j in range(n):  
            if type(rows[j][i]) == type('string'):
                rows[j][i-1] -= mn
                rows[j][i] = 'f' + str(mn)
                rows[j][i+1] += mn
    
    final_moves = []
    for i in range(len(rows[0])):
        if type(rows[0][i]) == type('string'):
            final_moves.append(rows[0][i])
            continue

        for j in range(n):
            if rows[j][i] > m/2:
                final_moves = final_moves + ['-r' + str(j)] * (m - rows[j][i])
            else:
                final_moves = final_moves + ['r' + str(j)] * (rows[j][i])

    final_final_moves = []
    i = 0
    while i < len(final_moves):
        if i != len(final_moves) - 1 and ('f' in final_moves[i]) and final_moves[i] == final_moves[i+1]:
            i += 2
        else:
            final_final_moves.append(final_moves[i])
            i += 1
    
    return final_final_moves

def ex_red(moves):
    ex = expand(moves)
    return shrink(ex)

print(len(ex_red(moves)))

479
